<a href="https://colab.research.google.com/github/kanvaudupa-bioinfo/Bioinfo-lab/blob/main/1RV23BT033_BLAST%2BGlobal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.9 MB/s eta 0:00:00


Retrieve BLAST of query sequence (top 2 and bottom 2 results), run global alignment between query and results sequence
(Global align BLAST)

In [2]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import Entrez
import io

def get_sequence_from_ncbi(accession_id, email="your.email@example.com"):
    """
    Fetches a nucleotide sequence from NCBI using an accession ID.

    Args:
        accession_id (str): The NCBI accession ID.
        email (str): Your email address (required by NCBI).

    Returns:
        Seq: The fetched sequence object, or None if an error occurs.
    """
    Entrez.email = email
    try:
        handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="fasta", retmode="text")
        fasta_data = handle.read()
        handle.close()

        # Parse the FASTA data to get the sequence
        # We can use SeqIO for more robust parsing if needed, but for a single sequence,
        # simple string manipulation is sufficient.
        lines = fasta_data.strip().split('\n')
        if len(lines) > 1:
            sequence = "".join(lines[1:])
            return Seq(sequence)
        else:
            print(f"Could not retrieve sequence for accession ID: {accession_id}")
            return None

    except Exception as e:
        print(f"An error occurred while fetching sequence from NCBI: {e}")
        return None


def perform_ncbi_blast_and_global_align(sequence, num_top_bottom_blast=2, num_align_results=1):
    """
    Performs a BLAST search using NCBIWWW for a given sequence,
    selects the top and bottom results, and then performs global alignment
    between the query and those subject sequences.

    Args:
        sequence (Seq): The input nucleotide sequence as a Bio.Seq object.
        num_top_bottom_blast (int): The number of top and bottom BLAST results to display.
        num_align_results (int): The maximum number of global alignments to display per subject.
    """
    query_seq = sequence
    blast_results_list = []

    print("=== NCBI BLAST (nucleotide blast) ===")
    try:
        result_handle = NCBIWWW.qblast("blastn", "nt", query_seq)
        blast_records = NCBIXML.parse(result_handle)

        for blast_record in blast_records:
            print(f"Query: {blast_record.query}")
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    # Store relevant information including the E-value for sorting
                    blast_results_list.append({
                        "alignment_title": alignment.title,
                        "alignment_length": alignment.length,
                        "e_value": hsp.expect,
                        "query_start": hsp.query_start,
                        "query_end": hsp.query_end,
                        "subject_start": hsp.sbjct_start,
                        "subject_end": hsp.sbjct_end,
                        "score": hsp.score,
                        "identities": hsp.identities,
                        "gaps": hsp.gaps,
                        "query_snippet": hsp.query[0:75] + "...",
                        "match_snippet": hsp.match[0:75] + "...",
                        "sbjct_snippet": hsp.sbjct[0:75] + "...",
                        "subject_seq": Seq(hsp.sbjct) # Store subject sequence for alignment
                    })

        result_handle.close()

    except Exception as e:
        print(f"An error occurred during BLAST: {e}")
        return # Exit if BLAST fails

    # Sort BLAST results by E-value (lower is better)
    blast_results_list.sort(key=lambda x: x['e_value'])

    # Select top and bottom results
    top_results = blast_results_list[:num_top_bottom_blast]
    bottom_results = blast_results_list[-num_top_bottom_blast:] if len(blast_results_list) > num_top_bottom_blast else [] # Handle cases with fewer results

    print(f"\n=== Top {num_top_bottom_blast} BLAST Results ===")
    blast_subject_sequences = {}
    for result in top_results:
        print(f"  Alignment: {result['alignment_title']}")
        print(f"  Length: {result['alignment_length']}")
        print(f"  E-value: {result['e_value']}")
        print(f"  Query start: {result['query_start']}, Query end: {result['query_end']}")
        print(f"  Subject start: {result['subject_start']}, Subject end: {result['subject_end']}")
        print(f"  Score: {result['score']}")
        print(f"  Identities: {result['identities']}")
        print(f"  Gaps: {result['gaps']}")
        print(f"  Query: {result['query_snippet']}")
        print(f"  Match: {result['match_snippet']}")
        print(f"  Sbjct: {result['sbjct_snippet']}")
        print("-" * 80)
        blast_subject_sequences[result['alignment_title']] = result['subject_seq']


    print(f"\n=== Bottom {num_top_bottom_blast} BLAST Results ===")
    for result in bottom_results:
        print(f"  Alignment: {result['alignment_title']}")
        print(f"  Length: {result['alignment_length']}")
        print(f"  E-value: {result['e_value']}")
        print(f"  Query start: {result['query_start']}, Query end: {result['query_end']}")
        print(f"  Subject start: {result['subject_start']}, Subject end: {result['subject_end']}")
        print(f"  Score: {result['score']}")
        print(f"  Identities: {result['identities']}")
        print(f"  Gaps: {result['gaps']}")
        print(f"  Query: {result['query_snippet']}")
        print(f"  Match: {result['match_snippet']}")
        print(f"  Sbjct: {result['sbjct_snippet']}")
        print("-" * 80)
        # Add bottom results to the subject sequences for alignment
        blast_subject_sequences[result['alignment_title']] = result['subject_seq']


    # Perform Global Alignment with selected BLAST Subject Sequences
    print("\n=== Global Alignment with Selected BLAST Subject Sequences ===")
    if blast_subject_sequences:
        for title, subject_seq in blast_subject_sequences.items():
            print(f"Global Alignment of Query with: {title}")
            # Use globalms for scoring with match, mismatch, open_gap_score, extend_gap_score
            global_alignments = pairwise2.align.globalms(query_seq, subject_seq, 5, -4, -10, -0.5) # Example scoring
            if global_alignments:
                # Limit the number of displayed alignments
                for i, alignment in enumerate(global_alignments):
                    if i < num_align_results:
                        print(format_alignment(*alignment))
                    else:
                        break
            else:
                print("No global alignments found.")
            print("-" * 80)
    else:
        print("No subject sequences retrieved from BLAST to perform global alignment.")


# Example usage:
input_accession_id = "AF315290.1" # Replace with your desired accession ID
fetched_sequence = get_sequence_from_ncbi(input_accession_id)

if fetched_sequence:
    perform_ncbi_blast_and_global_align(fetched_sequence, num_top_bottom_blast=2, num_align_results=1)
else:
    print(f"Could not retrieve sequence for accession ID: {input_accession_id}. Cannot proceed with analysis.")

/usr/local/lib/python3.12/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


=== NCBI BLAST (nucleotide blast) ===
Query: No definition line

=== Top 2 BLAST Results ===
  Alignment: gi|13649766|gb|AF315290.1| Mus musculus collagen-like Alzheimer amyloid plaque component precursor type I mRNA, complete cds
  Length: 2298
  E-value: 0.0
  Query start: 1, Query end: 2298
  Subject start: 1, Subject end: 2298
  Score: 4596.0
  Identities: 2298
  Gaps: 0
  Query: CCCGGCGCCACACAGTCCCCGGCCGGAGGGTGCTTTTCACTCCTAGCTGGAAGGGGAGAAAGAATCTGGAGGACG...
  Match: |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
  Sbjct: CCCGGCGCCACACAGTCCCCGGCCGGAGGGTGCTTTTCACTCCTAGCTGGAAGGGGAGAAAGAATCTGGAGGACG...
--------------------------------------------------------------------------------
  Alignment: gi|350276162|ref|NM_029838.4| Mus musculus collagen, type XXV, alpha 1 (Col25a1), transcript variant 1, mRNA
  Length: 7421
  E-value: 0.0
  Query start: 1, Query end: 2298
  Subject start: 373, Subject end: 2670
  Score: 4581.0
  Identities: 2295
  Gaps: 0
  Quer